# Visualização Doc2Vec com TensorBoard

## 1. Definimos a função para leitura e preprocessamento do texto

In [1]:
import gensim
import pandas as pd
import smart_open
import random
from tensorboard_helper import CreateTensorboardData

# read data
dataframe = pd.read_csv('movie_plots.csv')
dataframe

,MovieID,Titles,Plots,Genres
0,1,Toy Story (1995),A little boy named Andy loves to be in his roo...,animation
1,2,Jumanji (1995),When two kids find and play a magical board ga...,fantasy
2,3,Grumpier Old Men (1995),Things don't seem to change much in Wabasha Co...,comedy
3,6,Heat (1995),Hunters and their prey--Neil and his professio...,action
4,7,Sabrina (1995),An ugly duckling having undergone a remarkable...,romance
5,9,Sudden Death (1995),Some terrorists kidnap the Vice President of t...,action
6,10,GoldenEye (1995),James Bond teams up with the lone survivor of ...,action
7,15,Cutthroat Island (1995),"Morgan Adams and her slave, William Shaw, are ...",action
8,17,Sense and Sensibility (1995),"When Mr. Dashwood dies, he must leave the bulk...",romance
9,18,Four Rooms (1995),This movie features the collaborative director...,comedy


Definimos uma função de leitura dos documentos de treinamento, preprocessamos os documentos, utilizando a ferramenta simple_preprocess de gensim. Para treinar o modelo, temos que associas Tag/number para cada documento de treinamento. (Neste caso é simplemente i=0,1,2...)

In [2]:
def read_corpus(documents):
    for i, plot in enumerate(documents):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(plot, max_len=30), [i])

In [3]:
train_corpus = list(read_corpus(dataframe.Plots))

Observamos algunos dados de treinamento (corpus)

In [4]:
train_corpus[:2]

[TaggedDocument(words=['little', 'boy', 'named', 'andy', 'loves', 'to', 'be', 'in', 'his', 'room', 'playing', 'with', 'his', 'toys', 'especially', 'his', 'doll', 'named', 'woody', 'but', 'what', 'do', 'the', 'toys', 'do', 'when', 'andy', 'is', 'not', 'with', 'them', 'they', 'come', 'to', 'life', 'woody', 'believes', 'that', 'he', 'has', 'life', 'as', 'toy', 'good', 'however', 'he', 'must', 'worry', 'about', 'andy', 'family', 'moving', 'and', 'what', 'woody', 'does', 'not', 'know', 'is', 'about', 'andy', 'birthday', 'party', 'woody', 'does', 'not', 'realize', 'that', 'andy', 'mother', 'gave', 'him', 'an', 'action', 'figure', 'known', 'as', 'buzz', 'lightyear', 'who', 'does', 'not', 'believe', 'that', 'he', 'is', 'toy', 'and', 'quickly', 'becomes', 'andy', 'new', 'favorite', 'toy', 'woody', 'who', 'is', 'now', 'consumed', 'with', 'jealousy', 'tries', 'to', 'get', 'rid', 'of', 'buzz', 'then', 'both', 'woody', 'and', 'buzz', 'are', 'now', 'lost', 'they', 'must', 'find', 'way', 'to', 'get',

## 2. Treinando o modelo doc2vec

Inicializamos o modelo Doc2Vec com um vetor size=45 e rtainar 100 iterações. Minima repetições por palavra min_count=2 
A precisão do modelo pode melhorar incrementando o número de iterações, mas geralmente incrementa o tempo de treinamento. Para pequenos documentos (como este caso, podese aumentar o número de iterações)

In [15]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

5167084

Agora salvamos, o documento de vetores embedding por cada pelo tag.

In [16]:
tensor_file = 'doc_tensor.w2v'
model.save_word2vec_format(tensor_file, doctag_vec=True, word_vec=False)
key_vectors = gensim.models.KeyedVectors.load_word2vec_format(tensor_file, binary=False)

In [17]:
plots_id = []
vectors = []
for plot_id in key_vectors.index2word:
    plots_id.append(plot_id)
    vectors.append(key_vectors[plot_id])

## 3. Preparando os arquivos de entrada para TensorBoard

TensorBoard precisa 2 arquivos de entrada. Uma contendo os vetores embedding e a outra com os metadados relevantes. Utilizamos os arquivos de ajuda para criar estes arquivos.

In [18]:
metadatos=[plots_id]
CreateTensorboardData(tensor_filename="movie_plot", 
                      vectors=vectors, 
                      metadatos=[plots_id])

Arquivo com o Tensor 2D foi salvado em: movie_plot_tensor.tsv
Arquivo com o Tensor de metadatos foi salvado em: movie_plot_metadata.tsv


Baixar arquivos de vetores : <a href="movie_plot_tensor.tsv" download="w3logo"> movie_plot_tensor.tsv </a>

Baixar arquivos com os metadatos: <a href="movie_plot_metadata.tsv" download="w3logo"> movie_plot_metadata.tsv </a>

In [19]:
metadatos=[dataframe.Titles.values, dataframe.Genres.values]

CreateTensorboardData(tensor_filename="movie_plot", 
                      vectors=vectors, 
                      metadatos=metadatos,
                      colnames=["Titles","Genres"])

Arquivo com o Tensor 2D foi salvado em: movie_plot_tensor.tsv
Arquivo com o Tensor de metadatos foi salvado em: movie_plot_metadata.tsv


Baixar arquivos de vetores : <a href="movie_plot_tensor.tsv" download="w3logo"> movie_plot_tensor.tsv </a>

Baixar arquivos com os metadatos: <a href="movie_plot_metadata.tsv" download="w3logo"> movie_plot_metadata.tsv </a>

## 4. Usando TensorBoard

**Análise de componentes principais**: o PCA tem como objetivo explorar a estrutura global dos dados e acabar por perder as semelhanças locais entre os vizinhos. Ele maximiza a variância total no subespaço dimensional inferior e, portanto, muitas vezes preserva as distâncias pairwise maiores melhores do que as menores.

**T-SNE**: A idéia de T-SNE é colocar os vizinhos locais próximos uns dos outros e ignorar quase completamente a estrutura global. É útil para explorar bairros locais e encontrar clusters locais. Mas as tendências globais não são representadas com precisão e a separação entre diferentes grupos muitas vezes não é preservada, como observaremos.

Abrir http://projector.tensorflow.org/ e colocar o vetores do arquivo **movie_plot_vector.tsv** e os **metadatos movie_plot_metadata.tsv**